In [146]:
import os
from pathlib import Path
import matlab.engine
import numpy as np
import shutil
import tempfile
import json

# Base directory (adjust to your setup)
base_dir = Path("./")

 # --- Start MATLAB Engine ---
eng = matlab.engine.start_matlab()

 # --- Add SCOPE path to MATLAB's search path ---
scope_path = './SCOPE'  # e.g., '/home/user/Documents/SCOPE'
eng.addpath(scope_path, nargout=0)

# MODTRAN paths
modtran_dir = base_dir / "MODTRAN5"
modtran_exe = modtran_dir / "bin" / "Mod5_mac.exe" 
modtran_tp5_template = modtran_dir / "HyPlant-FLUO_Modtran5_base_v1.tp5"

# SCOPE paths
scope_dir = base_dir / "SCOPE"
scope_main = scope_dir / "SCOPE.m"
scope_wrapper = scope_dir / "run_scope_wrapper.m"

# Output directory
output_dir = base_dir / "synthetic_dataset"
output_dir.mkdir(exist_ok=True)

print(f"Checking path: {scope_dir.resolve()}")
print(f"Directory exists: {scope_dir.exists()}")
print(f"Directory contents: {list(scope_dir.glob('*')) if scope_dir.exists() else 'N/A'}")


Checking path: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE
Directory exists: True
Directory contents: [PosixPath('SCOPE/.DS_Store'), PosixPath('SCOPE/run_scope_wrapper_json.m'), PosixPath('SCOPE/input'), PosixPath('SCOPE/output'), PosixPath('SCOPE/docs'), PosixPath('SCOPE/soltir_tp7.m'), PosixPath('SCOPE/SCOPE.m'), PosixPath('SCOPE/README.md'), PosixPath('SCOPE/set_parameter_filenames.csv'), PosixPath('SCOPE/bug_reports.txt'), PosixPath('SCOPE/GNU_General_Public_Licence.txt'), PosixPath('SCOPE/.git'), PosixPath('SCOPE/.readthedocs.yaml'), PosixPath('SCOPE/SCOPE.exe'), PosixPath('SCOPE/src')]


In [ ]:
import csv
import os
import shutil
import json
import matlab.engine
from pathlib import Path
import numpy as np

class SCOPEWrapper:
    def __init__(self):
        self.base_dir = Path.cwd()  # More robust; use current working directory
        self.scope_dir = self.base_dir / "SCOPE"
        self.original_input = self.scope_dir / "input" / "input_data.csv"
        self.default_params = self._get_default_parameters()
        self.eng = None  # Initialize engine to None

    def _get_default_parameters(self):
        """Comprehensive default parameters for SCOPE (as before)."""
        return {
            # PROSPECT
            'Cab': 40.0, 'Cca': 10.0, 'Cdm': 0.012, 'Cw': 0.009,
            'Cs': 0.0, 'Cant': 1.0, 'Cp': 0.0, 'Cbc': 0.0,
            'N': 1.5, 'rho_thermal': 0.01, 'tau_thermal': 0.01,
            
            # Leaf Biochemical
            'Vcmax25': 60.0, 'BallBerrySlope': 8.0, 'BallBerry0': 0.01,
            'Type': 0.0, 'kV': 0.64, 'Rdparam': 0.015, 'Kn0': 2.48,
            'Knalpha': 2.83, 'Knbeta': 0.114,
            
            # Magnani
            'Tyear': 15.0, 'beta': 0.51, 'kNPQs': 0.0, 
            'qLs': 1.0, 'stressfactor': 1.0,
            
            # Fluorescence
            'fqe': 0.01,
            
            # Soil
            'spectrum': 1.0, 'rss': 500.0, 'rs_thermal': 0.06,
            'cs': 1180.0, 'rhos': 1800.0, 'lambdas': 1.55,
            'SMC': 25.0, 'BSMBrightness': 0.5, 'BSMlat': 25.0,
            'BSMlon': 45.0,
            
            # Canopy
            'LAI': 3.0, 'hc': 2.0, 'LIDFa': -0.35, 'LIDFb': -0.15,
            'leafwidth': 0.1, 'Cv': 1.0, 'crowndiameter': 1.0,
            
            # Meteo
            'z': 5.0, 'Rin': 800.0, 'Ta': 20.0, 'Rli': 300.0,
            'p': 970.0, 'ea': 15.0, 'u': 2.0, 'Ca': 410.0, 'Oa': 209.0,
            
            # Aerodynamic
            'zo': 0.25, 'd': 1.34, 'Cd': 0.3, 'rb': 10.0,
            'CR': 0.35, 'CD1': 20.6, 'Psicor': 0.2,
            'CSSOIL': 0.01, 'rbs': 10.0, 'rwc': 0.0,
            
            # Timeseries
            'startDOY': 20060618.0, 'endDOY': 20300101.0,
            'LAT': 51.55, 'LON': 5.55, 'timezn': 1.0,
            
            # Angles
            'tts': 35.0, 'tto': 0.0, 'psi': 0.0
        }

    def __enter__(self):
        """Start MATLAB engine and add SCOPE path."""
        self.eng = matlab.engine.start_matlab('-nodisplay')  # Add -nodisplay for no GUI
        self.eng.addpath(self.eng.genpath(str(self.scope_dir)), nargout=0)  # Use genpath
        self.eng.cd(str(self.scope_dir), nargout=0)  # Change to SCOPE directory
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Stop MATLAB engine and restore input file."""
        if self.eng:
            self.eng.quit()

         # Restore original file (important for repeated runs)
        if hasattr(self, 'temp_backup') and self.temp_backup.exists():
            try:
                if self.original_input.exists():
                    os.remove(str(self.original_input))
                shutil.move(str(self.temp_backup), str(self.original_input))
            except Exception as e:
                print(f"Error restoring original input file: {e}")



    def _prepare_input(self, csv_content):
        """Write input directly to SCOPE's input directory"""
        input_dir = self.scope_dir / "input"
        input_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
        temp_file = input_dir / "input_data.csv"

        # Backup original file if it exists
        if self.original_input.exists():
            self.temp_backup = self.original_input.with_name("input_data.backup.csv")
            if self.temp_backup.exists():
                os.remove(str(self.temp_backup)) # Delete previous backup to avoid errors.
            shutil.move(str(temp_file), str(self.temp_backup))

        # Write new content
        with open(temp_file, 'w') as f:
            f.write(csv_content)
        return temp_file
            

    def _generate_csv_content(self, params):
        """Generate CSV content with exact SCOPE formatting (same as before)."""
        sections = [
            ('PROSPECT', [
                'Cab', 'Cca', 'Cdm', 'Cw', 'Cs', 'Cant', 
                'Cp', 'Cbc', 'N', 'rho_thermal', 'tau_thermal'
            ]),
            ('Leaf_Biochemical', [
                'Vcmax25', 'BallBerrySlope', 'BallBerry0', 'Type',
                'kV', 'Rdparam', 'Kn0', 'Knalpha', 'Knbeta'
            ]),
            ('Leaf_Biochemical_magnani', [
                'Tyear', 'beta', 'kNPQs', 'qLs', 'stressfactor'
            ]),
            ('Fluorescence', ['fqe']),
            ('Soil', [
                'spectrum', 'rss', 'rs_thermal', 'cs', 'rhos',
                'lambdas', 'SMC', 'BSMBrightness', 'BSMlat', 'BSMlon'
            ]),
            ('Canopy', [
                'LAI', 'hc', 'LIDFa', 'LIDFb', 'leafwidth',
                'Cv', 'crowndiameter'
            ]),
            ('Meteo', [
                'z', 'Rin', 'Ta', 'Rli', 'p', 'ea', 'u', 'Ca', 'Oa'
            ]),
            ('Aerodynamic', [
                'zo', 'd', 'Cd', 'rb', 'CR', 'CD1', 'Psicor',
                'CSSOIL', 'rbs', 'rwc'
            ]),
            ('timeseries', [
                'startDOY', 'endDOY', 'LAT', 'LON', 'timezn'
            ]),
            ('Angles', ['tts', 'tto', 'psi'])
        ]
        content = []
        for section, parameters in sections:
            content.append(f"{section},")
            for param in parameters:
                value = params.get(param, self.default_params[param])  # Use get with default
                content.append(f"{param},{float(value)}")
            content.append(",")
            
        return "\r\n".join(content) + "\r\n"



    def _find_latest_output(self):
        """Find the newest output directory (same as before)."""
        output_parent = self.scope_dir / "output"
        runs = sorted(output_parent.glob("example_run_*"), key=os.path.getmtime, reverse=True)
        if not runs:
            raise FileNotFoundError("No SCOPE output directories found.")
        return runs[0]

    def _read_data(self, filepath):
        """Reads data from a file, handling both CSV and TXT."""
        if filepath.suffix.lower() == '.csv':
            try:
                # Skip header row, handle comma as decimal
                return np.loadtxt(filepath, delimiter=',', skiprows=1, dtype=float).tolist()
            except Exception as e:
                print(f"Error reading CSV file {filepath}: {e}")
                return None  # Or [] or {} depending on context
        elif filepath.suffix.lower() == '.txt':
            try:
                wavelengths = []
                with open(filepath, 'r') as f:
                    for line in f:
                        # Split each line by spaces and convert to floats
                        values = [float(value) for value in line.split()]
                        wavelengths.extend(values)
                return wavelengths
            except Exception as e:
                print(f"Error reading TXT file {filepath}: {e}")
                return None
        else:
            print(f"Unsupported file type: {filepath.suffix}")
            return None
    
    
    def _read_parameters(self, filepath):
        """Reads parameters from a CSV file, handling section headers."""
        params = {}
        with open(filepath, 'r') as f:
            reader = csv.reader(f)
            current_section = None
            for row in reader:
                if not row:  # Skip empty lines
                    continue
                if row[0].endswith(','):  # Section header
                    current_section = row[0].strip(',')
                elif len(row) >= 2 and current_section:
                    param_name = row[0].strip()
                    param_value = row[1].strip()
                    # Convert to float if possible, otherwise keep as string
                    try:
                        param_value = float(param_value)
                    except ValueError:
                        pass  # Keep as string if conversion fails
                    params[f"{current_section}.{param_name}"] = param_value
        return params


    def generate_results_json(self, output_dir: Path) -> Path:
        """Generate a comprehensive results.json from all SCOPE outputs."""
        results = {}
        # Process the Parameters subdirectory
        params_dir = output_dir / "Parameters"
        if params_dir.exists() and params_dir.is_dir():
            for param_file in params_dir.glob("*.csv"):
                if param_file.name.startswith("filenames"):
                    # Special handling for filenames
                    filenames = {}
                    with open(param_file, 'r') as f:
                        reader = csv.reader(f)
                        for row in reader:
                            if len(row) >= 2:
                                filenames[row[0].strip()] = row[1].strip()
                    results['filenames'] = filenames
                elif param_file.name.startswith("input_data"):
                    # Handle input_data separately
                    results['input_data'] = self._read_parameters(param_file)
                elif param_file.name.startswith("setoptions"):
                    # Handle setoptions separately
                    results['setoptions'] = self._read_parameters(param_file)

        # Read data files (CSV and TXT) in the main output directory
        for file in output_dir.iterdir():
            if file.is_file():  # Only process files
                data = self._read_data(file)
                if data is not None:
                    results[file.stem] = data
        
        
        json_path = output_dir / "results.json"
        with open(json_path, 'w') as f:
            json.dump(results, f, indent=2)

        return json_path

    def _write_scope_csv(self, params):
        """Write parameters directly to SCOPE's input_data.csv"""
        input_path = self.scope_dir / "input" / "input_data.csv"
        
        # Generate CSV content using your existing method
        csv_content = self._generate_csv_content(params)  
        
        # Write directly to SCOPE's expected input location
        with open(input_path, 'w', newline='') as f:
            f.write(csv_content)

    def run(self, params):
      """Run SCOPE with the given parameters and return parsed output."""
      try:
          # Write parameters to input file.
          self._write_scope_csv(params)

          # Run SCOPE (ensure the MATLAB script is in the MATLAB path).
          # The `nargout=0` is crucial to prevent hanging.
          self.eng.run_scope_wrapper_json(nargout=0)

          # Find the output directory.
          output_dir = self._find_latest_output()

          # Process the output and return it.
          json_path = self.generate_results_json(output_dir)
          return json_path


      except matlab.engine.MatlabExecutionError as e:
          print(f"MATLAB Error: {str(e)}")
          raise  # Re-raise to halt execution
      except FileNotFoundError as e:
          print(f"File Not Found Error: {str(e)}")
          raise
      except Exception as e:
          print(f"An unexpected error occurred: {str(e)}")
          raise

In [154]:
import pandas as pd

if __name__ == "__main__":
    #Diagnostic output
    print("\n=== Path Diagnostics ===")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Checking for 'SCOPE' in: {Path.cwd()}")

    try:
        with SCOPEWrapper() as scope:
            print("\n=== Running Test Simulation ===")
            # Example parameters (can be adjusted)
            test_params = {
                'Cab': 45.0,
                'LAI': 3.2,
                'hc': 1.5,
                'tts': 35.0,
                'Rin': 800.0
            }
            results_path = scope.run(test_params)
            print("Success!")
            # You can now load and inspect the results.json here if needed
            with open(results_path, 'r') as f:
                results = json.load(f)
            print("\n=== Results ===")
            print(json.dumps(results, indent=2))
    except Exception as e:
        print(f"Error during SCOPE execution: {e}")


=== Path Diagnostics ===
Current working directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project
Checking for 'SCOPE' in: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project

=== Running Test Simulation ===

 Do not quench your inspiration and your imagination; do not become the slave of your model (Vincent van Gogh). 
simulation 1 of 1 start now 
Elapsed time is 0.420382 seconds.
File Not Found Error: No SCOPE output directories found.
Error during SCOPE execution: No SCOPE output directories found.


In [ ]:
from collections import defaultdict
import csv
import itertools
import os
import shutil
import json
import matlab.engine
from pathlib import Path
import numpy as np
import pandas as pd
import struct
import re  # Import the regular expression module


class SCOPEWrapperMultiRun:
    def __init__(self, scalar_files, spectral_files):
        self.base_dir = Path.cwd()
        self.scope_dir = self.base_dir / "SCOPE"
        self.original_input = self.scope_dir / "input" / "input_data.csv"
        self.default_params = self._get_default_parameters()
        self.eng = None
        self.setoptions = self._get_default_setoptions()  # Load default setoptions
        self.active_setoptions = self._get_default_setoptions()  # Track active setoptions
        self.scalar_files = scalar_files
        self.spectral_files = spectral_files
        


    def _get_default_setoptions(self):
        """Loads default setoptions from setoptions.csv"""
        setoptions_path = self.scope_dir / "input" / "setoptions.csv"
        setoptions = {}
        with open(setoptions_path, "r") as f:
            reader = csv.reader(f)
            for row in reader:
                if row and len(row) >= 2:  # Ensure row is not empty and has key-value
                    key = row[0].strip()
                    value = row[1].strip()
                    try:
                        # Convert to int if possible
                        value = int(value)
                    except ValueError:
                        pass  # Keep as string if conversion fails
                    setoptions[key] = value
        return setoptions

    def _get_default_parameters(self):
        """Comprehensive default parameters for SCOPE."""
        return {
            # PROSPECT
            "Cab": 40.0,
            "Cca": 10.0,
            "Cdm": 0.012,
            "Cw": 0.009,
            "Cs": 0.0,
            "Cant": 1.0,
            "Cp": 0.0,
            "Cbc": 0.0,
            "N": 1.5,
            "rho_thermal": 0.01,
            "tau_thermal": 0.01,
            # Leaf Biochemical
            "Vcmax25": 60.0,
            "BallBerrySlope": 8.0,
            "BallBerry0": 0.01,
            "Type": 0.0,
            "kV": 0.64,
            "Rdparam": 0.015,
            "Kn0": 2.48,
            "Knalpha": 2.83,
            "Knbeta": 0.114,
            # Magnani
            "Tyear": 15.0,
            "beta": 0.51,
            "kNPQs": 0.0,
            "qLs": 1.0,
            "stressfactor": 1.0,
            # Fluorescence
            "fqe": 0.01,
            # Soil
            "spectrum": 1.0,
            "rss": 500.0,
            "rs_thermal": 0.06,
            "cs": 1180.0,
            "rhos": 1800.0,
            "lambdas": 1.55,
            "SMC": 25.0,
            "BSMBrightness": 0.5,
            "BSMlat": 25.0,
            "BSMlon": 45.0,
            # Canopy
            "LAI": 3.0,
            "hc": 2.0,
            "LIDFa": -0.35,
            "LIDFb": -0.15,
            "leafwidth": 0.1,
            "Cv": 1.0,
            "crowndiameter": 1.0,
            # Meteo
            "z": 5.0,
            "Rin": 800.0,
            "Ta": 20.0,
            "Rli": 300.0,
            "p": 970.0,
            "ea": 15.0,
            "u": 2.0,
            "Ca": 410.0,
            "Oa": 209.0,
            # Aerodynamic
            "zo": 0.25,
            "d": 1.34,
            "Cd": 0.3,
            "rb": 10.0,
            "CR": 0.35,
            "CD1": 20.6,
            "Psicor": 0.2,
            "CSSOIL": 0.01,
            "rbs": 10.0,
            "rwc": 0.0,
            # Timeseries
            "startDOY": 20060618.0,
            "endDOY": 20300101.0,
            "LAT": 51.55,
            "LON": 5.55,
            "timezn": 1.0,
            # Angles
            "tts": 35.0,
            "tto": 0.0,
            "psi": 0.0,
        }

    def __enter__(self):
        """Start MATLAB engine and add SCOPE path."""
        self.eng = matlab.engine.start_matlab("-nodisplay")
        self.eng.addpath(self.eng.genpath(str(self.scope_dir)), nargout=0)
        self.eng.cd(str(self.scope_dir), nargout=0)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Stop MATLAB engine and restore input file."""
        if self.eng:
            self.eng.quit()

        if hasattr(self, "temp_backup") and self.temp_backup.exists():
            try:
                if self.original_input.exists():
                    os.remove(str(self.original_input))
                shutil.move(str(self.temp_backup), str(self.original_input))
            except Exception as e:
                print(f"Error restoring original input file: {e}")

    def _prepare_input(self, csv_content):
        """Write input directly to SCOPE's input directory, backing up the original."""
        input_dir = self.scope_dir / "input"
        input_dir.mkdir(parents=True, exist_ok=True)
        temp_file = input_dir / "input_data.csv"

        # Backup original file if it exists
        if self.original_input.exists():
            self.temp_backup = self.original_input.with_name("input_data.backup.csv")
            if self.temp_backup.exists():
                os.remove(str(self.temp_backup))
            shutil.move(str(self.original_input), str(self.temp_backup))

        # Write new content
        with open(temp_file, "w") as f:
            f.write(csv_content)
        return temp_file

    def _generate_csv_content(self, params):
        """Generate CSV content with parameters in SCOPE's expected row-based format"""
        sections = [
            (
                "PROSPECT",
                [
                    "Cab",
                    "Cca",
                    "Cdm",
                    "Cw",
                    "Cs",
                    "Cant",
                    "Cp",
                    "Cbc",
                    "N",
                    "rho_thermal",
                    "tau_thermal",
                ],
            ),
            (
                "Leaf_Biochemical",
                [
                    "Vcmax25",
                    "BallBerrySlope",
                    "BallBerry0",
                    "Type",
                    "kV",
                    "Rdparam",
                    "Kn0",
                    "Knalpha",
                    "Knbeta",
                ],
            ),
            (
                "Leaf_Biochemical_magnani",
                ["Tyear", "beta", "kNPQs", "qLs", "stressfactor"],
            ),
            ("Fluorescence", ["fqe"]),
            (
                "Soil",
                [
                    "spectrum",
                    "rss",
                    "rs_thermal",
                    "cs",
                    "rhos",
                    "lambdas",
                    "SMC",
                    "BSMBrightness",
                    "BSMlat",
                    "BSMlon",
                ],
            ),
            (
                "Canopy",
                ["LAI", "hc", "LIDFa", "LIDFb", "leafwidth", "Cv", "crowndiameter"],
            ),
            ("Meteo", ["z", "Rin", "Ta", "Rli", "p", "ea", "u", "Ca", "Oa"]),
            (
                "Aerodynamic",
                [
                    "zo",
                    "d",
                    "Cd",
                    "rb",
                    "CR",
                    "CD1",
                    "Psicor",
                    "CSSOIL",
                    "rbs",
                    "rwc",
                ],
            ),
            ("timeseries", ["startDOY", "endDOY", "LAT", "LON", "timezn"]),
            ("Angles", ["tts", "tto", "psi"]),
        ]

        lines = []
        self.structured_params = defaultdict(dict)  # Track params with section keys
        
        for section_name, param_names in sections:
            lines.append(f"{section_name},")
            for param in param_names:
                val = params.get(param, self.default_params.get(param, 0.0))
                if not isinstance(val, (list, tuple)):
                    val = [val]
                # Store with section key (e.g., "PROSPECT.Cab": [30,40,50])
                self.structured_params[section_name][param] = val
                line = f"{param}," + ",".join(map(str, val))
                lines.append(line)
            lines.append(",")
        return "\r\n".join(lines)

    def _find_latest_output(self):
        """Find the most recent output directory with enhanced logging."""
        output_parent = self.scope_dir / "output"
        if not output_parent.exists():
            raise FileNotFoundError(f"Output directory {output_parent} not found.")

        runs = list(output_parent.glob("scope_data_*"))
        if not runs:
            raise FileNotFoundError("No SCOPE output directories found.")

        # Debugging: Print found directories
        print("Found output directories:")
        for run in runs:
            print(f" - {run.name}")

        latest_run = max(runs, key=os.path.getmtime)
        print(f"Selected latest output: {latest_run}")
        return latest_run

    def _read_data(self, filepath):
        """Reads numerical data from CSV, TXT, or BIN files."""
        if filepath.suffix.lower() == ".csv":
            try:
                # All spectral files: no headers, skip comments
                if any(x in filepath.name for x in ["spectrum", "Esun", "Esky", "Eout", "fluorescence", "sigmaF"]):
                    df = pd.read_csv(filepath, sep=",", header=None, comment='#')
                    # Convert to list of lists, where each sublist is one simulation's spectrum
                    return df.values.tolist()  # Now returns [n_simulations, n_wavelengths]
                # Scalar files: handle headers and comments
                else:
                    df = pd.read_csv(filepath, sep=",", header=0, comment='#')
                    return df.values.tolist()
            except Exception as e:
                print(f"Error reading {filepath}: {e}")
                return None

        elif filepath.suffix.lower() == ".txt":
            with open(filepath, "r") as f:
                # Special handling for wavelength files
                if filepath.name in ["wlS.txt", "wlF.txt"]:
                    # Read all values into a single flat list
                    values = []
                    for line in f:
                        line_values = [float(v) for v in line.strip().split()]
                        values.extend(line_values)
                    return [values]  # Wrap in list to match other formats
                
                # Regular TXT file handling
                return [[float(value) for value in re.split(r'\s+', line.strip())] 
                       for line in f]
            
        elif filepath.suffix.lower() == ".bin":
            try:
                with open(filepath, "rb") as f:
                    # Assuming 4-byte floats (float32)
                    buffer = f.read()
                    num_floats = len(buffer) // 4
                    data = struct.unpack(f"{num_floats}f", buffer)
                    return [list(data)] #Consistent output
            except Exception as e:
                print(f"Error reading BIN file {filepath}: {e}")
                return None

        else:
            print(f"Unsupported file type: {filepath.suffix}")
            return None

    def _read_parameters(self, filepath):
        """Reads parameters from CSV files, handling multiple values."""
        params = {}
        if filepath.suffix.lower() == ".csv":
            if "filenames" in filepath.name:
                with open(filepath, 'r') as f:
                    reader = csv.reader(f)
                    for row in reader:
                        if row and len(row) >= 2:
                            params[row[0].strip()] = row[1].strip()
            else:
                with open(filepath, 'r') as f:
                    reader = csv.reader(f)
                    current_section = None
                    for row in reader:
                        if not row:
                            continue
                        if row[0].endswith(","):
                            current_section = row[0].strip(",")
                        elif len(row) >= 1 and current_section:
                            param_name = row[0].strip()
                            values = [cell.strip() for cell in row[1:] if cell.strip()]
                            converted_values = []
                            for val in values:
                                try:
                                    converted_values.append(float(val))
                                except ValueError:
                                    converted_values.append(val)
                            if len(converted_values) == 1:
                                param_value = converted_values[0]
                            else:
                                param_value = converted_values
                            key = f"{current_section}.{param_name}"
                            params[key] = param_value
        elif filepath.suffix.lower() == ".txt":
            try:
                with open(filepath, 'r') as f:
                    params[filepath.stem] = f.read().strip()
            except Exception as e:
                print(f"Error reading TXT file {filepath}: {e}")
        return params

    def _read_setoptions(self, filepath):
        """Reads setoptions from a CSV file."""
        setoptions = {}
        with open(filepath, "r") as f:
            reader = csv.reader(f)
            for row in reader:
                if row and len(row) >= 2:  # Check for non-empty and sufficient length
                    key = row[0].strip()
                    value = row[1].strip()
                    try:
                        value = int(value)  # Try converting to integer
                    except ValueError:
                        pass
                    setoptions[key] = value
        return setoptions

    def generate_results_json(self, output_dir: Path) -> Path:
        """Generate results.json with properly formatted parameters."""
        results = {"scalar_outputs": {}}
        
        # 1. Create parameter mapping dictionary
        param_mapping = {}
        for section, params in self.structured_params.items():
            for param in params.keys():
                param_mapping[param] = f"{section}.{param}"

        # 2. Read and process parameter file
        param_file = output_dir / "pars_and_input_short.csv"
        if param_file.exists():
            param_df = pd.read_csv(param_file, comment='#')
            
            # Clean dataframe
            param_df = param_df.loc[:, ~param_df.columns.str.contains('^Unnamed')]
            param_df = param_df.drop(columns=['n_pars'], errors='ignore')
            
            # Rename columns using parameter mapping
            param_df = param_df.rename(columns=param_mapping)
            
            # Convert to list of dictionaries with proper types
            run_parameters = []
            for _, row in param_df.iterrows():
                param_dict = {}
                for col in param_df.columns:
                    # Convert numeric values to float, preserve others
                    try:
                        param_dict[col] = float(row[col])
                    except (ValueError, TypeError):
                        param_dict[col] = row[col]
                run_parameters.append(param_dict)
            
            # 3. Add fixed parameters with proper formatting
            fixed_params = {
                f"{section}.{param}": vals[0]  # Get scalar value
                for section, params in self.structured_params.items()
                for param, vals in params.items()
                if f"{section}.{param}" not in param_df.columns
            }
            
            # Add fixed params to all simulations
            for param_dict in run_parameters:
                param_dict.update(fixed_params)
            
            results.update({
                "run_parameters": run_parameters,
                "num_simulations": len(run_parameters),
                "input_parameters": self.structured_params,
                "setoptions": self.active_setoptions
            })
        else:
            results.update({
                "num_simulations": 0,
                "input_parameters": self.structured_params,
                "setoptions": self.active_setoptions
            })

        # Selected scalar outputs (now including .bin files)
        results["scalar_outputs"] = {}
        for key, filename in self.scalar_files.items():
            filepath = output_dir / filename
            if filepath.exists():
                data = self._read_data(filepath)
                if data is not None:
                    results["scalar_outputs"][key] = data

        # Add handling for binary (.bin) files if saveCSV = 0
        if self.setoptions.get("saveCSV", 1) == 0:  # Default to 1 if not present.
            bin_files = {
                "Rin": "Rin.bin",
                "Rli": "Rli.bin",
                "fluorescence_bin": "fluorescence.bin",  # Corrected key
                # Add other .bin files you want to include here
            }
            for key, filename in bin_files.items():
                filepath = output_dir / filename
                if filepath.exists():
                    data = self._read_data(filepath)
                    if data is not None:
                        results["scalar_outputs"][key] = data

        # Process Parameters directory, skip filenames
        params_dir = output_dir / "Parameters"
        if params_dir.exists() and params_dir.is_dir():
            for param_file in params_dir.glob("*"):
                if param_file.is_file():
                    # Skip filenames files
                    if param_file.name.startswith("filenames"):
                        continue
                    params = self._read_parameters(param_file)
                    results.setdefault("model_parameters", {}).update(params)

        results["spectral_outputs"] = {}
        for key, filename in self.spectral_files.items():
            filepath = output_dir / filename
            if filepath.exists():
                data = self._read_data(filepath)
                if data is not None:
                    results["spectral_outputs"][key] = data



        # Add wlF and wlS (wavelengths)
        wlf_file = output_dir / "wlF.txt"
        wls_file = output_dir / "wlS.txt"
        if wlf_file.exists():
            results["wlF"] = self._read_data(wlf_file)
        if wls_file.exists():
            results["wlS"] = self._read_data(wls_file)



        json_path = output_dir / "results.json"
        with open(json_path, "w") as f:
            json.dump(results, f, indent=2)

        return json_path
    
    def _generate_run_parameters(self):
        """Generate all parameter combinations for simulation=2, including fixed params."""
        param_groups = []
        # Collect all parameters (both varying and fixed)
        for section, params in self.structured_params.items():
            for param, values in params.items():
                key = f"{section}.{param}"
                param_groups.append((key, values))

        # Generate Cartesian product of all parameter values
        value_combinations = itertools.product(*[vals for (key, vals) in param_groups])
        
        # Create a dict for each combination
        run_parameters = []
        for combo in value_combinations:
            param_dict = {}
            for i, (key, _) in enumerate(param_groups):
                param_dict[key] = combo[i]
            run_parameters.append(param_dict)
        
        return run_parameters

    def _write_scope_csv(self, params, setoptions=None):
        """Write parameters directly to SCOPE's input_data.csv"""
        input_path = self.scope_dir / "input" / "input_data.csv"

        # Generate CSV content
        csv_content = self._generate_csv_content(params)

        # Write directly to SCOPE's expected input location
        with open(input_path, "w", newline="") as f:
            f.write(csv_content)

    def run(self, params, setoptions=None):
        """Run SCOPE, return results.json path.  Includes debugging prints."""

        try:
            if setoptions:
                self._update_setoptions(setoptions)  # Update setoptions.csv
            # Debugging: Print parameters before CSV generation
            print("=== Parameters before CSV generation ===")
            print(json.dumps(params, indent=2))

            # Prepare and write input data
            csv_content = self._generate_csv_content(params)
            print("\n=== Generated CSV Content ===")
            print(csv_content)  # Print the generated CSV content

            input_file = self._prepare_input(csv_content)

            # Run SCOPE
            self.eng.run_scope_wrapper_json(nargout=0)

            # Find output and generate results.json
            output_dir = self._find_latest_output()
            json_path = self.generate_results_json(output_dir)

            return json_path

        except matlab.engine.MatlabExecutionError as e:
            print(f"MATLAB Error: {str(e)}")
            raise
        except FileNotFoundError as e:
            print(f"File Not Found Error: {str(e)}")
            raise
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            raise

    def _update_setoptions(self, user_options):
        """Update setoptions while maintaining SCOPE's order and user values."""
        # Define required order of keys (without hardcoding values)
        required_keys = [
            "lite",
            "calc_fluor",
            "calc_planck",
            "calc_xanthophyllabs",
            "soilspectrum",
            "Fluorescence_model",
            "applTcorr",
            "verify",
            "saveCSV",
            "mSCOPE",
            "simulation",
            "calc_directional",
            "calc_vert_profiles",
            "soil_heat_method",
            "calc_rss_rbs",
            "MoninObukhov",
            "save_spectral"
        ]

        # Merge user options with defaults, prioritizing user values
        combined_options = {**self.active_setoptions, **user_options}
        
        # Build ordered list using required key order
        ordered_options = []
        for key in required_keys:
            value = combined_options.get(key, 0)  # Default to 0 if not provided
            ordered_options.append((str(value), key))

        # Write to CSV
        setoptions_path = self.scope_dir / "input" / "setoptions.csv"
        with open(setoptions_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(ordered_options)
        
        # Update internal tracking
        self.active_setoptions = combined_options




=== Path Diagnostics ===
Current working directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project
Checking for 'SCOPE' in: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project

=== Running Multi-Simulation Test ===
=== Parameters before CSV generation ===
{
  "Cab": [
    30.0,
    40.0,
    50.0
  ],
  "Cca": [
    10.0
  ],
  "Cdm": [
    0.012
  ],
  "Cw": [
    0.009
  ],
  "Cs": [
    0.0
  ],
  "Cant": [
    1.0
  ],
  "N": [
    1.5
  ],
  "Vcmax25": [
    60.0
  ],
  "BallBerrySlope": [
    8.0
  ],
  "BallBerry0": [
    0.01
  ],
  "LAI": [
    2.0,
    3.0,
    5.0
  ],
  "hc": [
    2.0
  ],
  "LIDFa": [
    -0.35
  ],
  "LIDFb": [
    -0.15
  ],
  "rss": [
    500.0
  ],
  "SMC": [
    25.0
  ],
  "Rin": [
    800.0
  ],
  "Ta": [
    20.0
  ],
  "Ca": [
    410.0
  ],
  "fqe": [
    0.01,
    0.03,
    0.06,
    0.1,
    0.2
  ],
  "tts": [
    35.0,
    0.0,
    0.6
  ],
  "tto": [
    0.0
  ],
  "psi": [
    0.0
  ]
}

In [ ]:


# Example Usage
if __name__ == "__main__":
    # Diagnostic output
    print("\n=== Path Diagnostics ===")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Checking for 'SCOPE' in: {Path.cwd()}")

    spectral_files = {
        #"Eout_spectrum": "Eout_spectrum.csv",
        #"Lo_spectrum": "Lo_spectrum.csv", 
        "fluorescence": "fluorescence.csv",
        #"sigmaF": "sigmaF.csv",
        "reflectance": "reflectance.csv",
    }
    
    scalar_files = {
        "aPAR": "aPAR.csv",
        #"Eout": "Eout.csv",
        #"Lo": "Lo.csv",
        #"Esun": "Esun.csv",
        #"Esky": "Esky.csv",
        #"fluxes": "fluxes.csv",
        #"rad": "radiation.csv",
        #"fluorescence_scalars": "fluorescence_scalars.csv",
    }

    try:
        with SCOPEWrapperMultiRun(scalar_files=scalar_files, spectral_files=spectral_files) as scope:
            print("\n=== Running Multi-Simulation Test ===")
            # Example parameters (can be adjusted)

            # Run a simulation with multiple LAI and Cab values
            multi_params = {
                # PROSPECT leaf optical properties
                "Cab": [30.0, 40.0, 50.0],      # Chlorophyll content (μg/cm²)
                "Cca": [10.0],                  # Carotenoid content (μg/cm²)
                "Cdm": [0.012],                 # Dry matter content (g/cm²)
                "Cw": [0.009],                  # Equivalent water thickness (cm)
                "Cs": [0.0],                    # Senescent material fraction (0-1)
                "Cant": [1.0],                  # Anthocyanin content (μg/cm²)
                "N": [1.5],                     # Leaf structure parameter (-)

                # Leaf biochemical parameters
                "Vcmax25": [60.0],              # Maximum carboxylation rate at 25°C (μmol/m²/s)
                "BallBerrySlope": [8.0],        # Ball-Berry stomatal conductance slope
                "BallBerry0": [0.01],           # Ball-Berry intercept (mol/m²/s)

                # Canopy structure
                "LAI": [2.0, 3.0, 5.0],        # Leaf Area Index (m²/m²)
                "hc": [2.0],                    # Canopy height (m)
                "LIDFa": [-0.35],               # Leaf angle distribution parameter a (-)
                "LIDFb": [-0.15],               # Leaf angle distribution parameter b (-)

                # Soil parameters
                "rss": [500.0],                 # Soil respiration rate (μmol/m²/s)
                "SMC": [25.0],                  # Soil moisture content (%)
                
                # Meteorology
                "Rin": [800.0],                 # Incoming shortwave radiation (W/m²)
                "Ta": [20.0],                   # Air temperature (°C)
                "Ca": [410.0],                  # Atmospheric CO₂ concentration (ppm)

                # Fluorescence
                "fqe": [0.01, 0.03, 0.06, 0.1, 0.2], # Fluorescence quantum efficiency (-)

                # Angles
                "tts": [35.0, 0.00, 0.60],                  # Solar zenith angle (degrees)
                "tto": [0.0],                   # Observer zenith angle (degrees)
                "psi": [0.0]                    # Relative azimuth angle (degrees)
            }

            my_setoptions = {
                "simulation": 2,  # Enable Lookup-Table mode
                "calc_fluor": 1,  # Calculate fluorescence
                "soil_heat_method": 2,  # Use soil net radiation fraction
                "saveCSV": 1,  # Output CSV files
                "calc_planck": 0,
                "calc_directional": 0,
                "calc_planck": 0,
                "calc_vert_profiles": 0,
                "calc_rss_rbs": 0,
                "soil_heat_method": 2,
                "save_spectral": 1,
                "calc_xanthophyllabs": 0,
                "applTcorr": 0,
                "MoninObukhov" : 0,
                "lite": 0,
                
            }

            results_path = scope.run(multi_params, my_setoptions)

            print("Success!")
            # You can now load and inspect the results.json here if needed
            with open(results_path, "r") as f:
                results = json.load(f)

            print("\n=== Results ===")

            print(
                f"Number of simulations in output: {results.get('num_simulations')}"
            ) 

    except Exception as e:
        print(f"Error during SCOPE execution: {e}")

In [136]:
class SCOPEHandler:
    """Improved SCOPE integration handler with MATLAB engine"""
    
    def __init__(self, scope_path):
        self.scope_path = Path(scope_path).resolve()
        self.eng = None
        self._validate_paths()

    def _validate_paths(self):
        required_files = {
            'main': self.scope_path / 'SCOPE.m',
            'wrapper': self.scope_path / 'run_scope_wrapper_json.m',
            'input_dir': self.scope_path / 'input'
        }
        
        missing = [k for k,v in required_files.items() if not v.exists()]
        if missing:
            raise FileNotFoundError(
                f"Missing required SCOPE files: {', '.join(missing)}\n"
                f"Verify SCOPE installation at: {self.scope_path}"
            )

    def __enter__(self):
        self.eng = matlab.engine.start_matlab('-nodisplay')
        self.eng.addpath(str(self.scope_path), nargout=0)
        self.eng.cd(str(self.scope_path), nargout=0)
        return self

    def __exit__(self, *exc):
        self.eng.quit()
        return False

    def run_simulation(self, parameters):
        """Execute SCOPE with parameter dictionary"""
        with NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
            json.dump(parameters, f)
            temp_json = f.name
        
        try:
            output = self.eng.run_scope_wrapper_json(
                temp_json, 
                str(self.scope_path / 'output' / 'results.json'),
                nargout=1
            )
            return self._parse_output(output)
        finally:
            os.unlink(temp_json)
            shutil.rmtree(self.scope_path / 'output', ignore_errors=True)

    def _parse_output(self, output):
        """Parse MATLAB output structure to Python dict"""
        return {
            'wavelength': np.array(output['wavelength']).ravel(),
            'reflectance': np.array(output['reflectance']).ravel(),
            'fluorescence': np.array(output['fluorescence']).ravel()
        }

In [137]:
import subprocess
from tempfile import NamedTemporaryFile


class MODTRANHandler:
    """MODTRAN simulation handler"""
    
    def __init__(self, modtran_path, template_file):
        self.modtran_path = Path(modtran_path).resolve()
        self.template = self.modtran_path / template_file
        self._validate_setup()

    def _validate_setup(self):
        if not (self.modtran_path / 'bin' / 'Mod5_mac.exe').exists():
            raise FileNotFoundError("MODTRAN executable not found")

    def run_simulation(self, params, case_id):
        """Execute MODTRAN simulation with parameters"""
        tp5_content = self._generate_tp5(params)
        
        with NamedTemporaryFile(mode='w', suffix='.tp5', delete=False) as f:
            f.write(tp5_content)
            tp5_path = Path(f.name)
        
        try:
            subprocess.run(
                [str(self.modtran_path / 'bin' / 'mod5_win64.exe'), str(tp5_path)],
                cwd=self.modtran_path,
                check=True,
                capture_output=True
            )
            return self._parse_output(tp5_path)
        finally:
            tp5_path.unlink()

    def _generate_tp5(self, params):
        """Generate TP5 file from template and parameters"""
        with open(self.template, 'r') as f:
            content = f.read()
        
        replacements = {
            '{H1}': f"{params['sensor_altitude']:.3f}",
            '{SZA}': f"{params['sza']:.1f}",
            '{VIS}': f"{params['visibility']:.2f}",
            # Add more parameter replacements as needed
        }
        
        for ph, val in replacements.items():
            content = content.replace(ph, val)
        
        return content

    def _parse_output(self, tp5_path):
        """Parse MODTRAN output files (.tp7)"""
        tp7_path = tp5_path.with_suffix('.tp7')
        
        # Implement actual parsing based on your MODTRAN output format
        return {
            'direct_transmittance': np.random.rand(100),
            'diffuse_transmittance': np.random.rand(100),
            'spherical_albedo': np.random.rand(100),
            'fluorescence_transmittance': np.random.rand(100)
        }


In [171]:
class SyntheticDataGenerator:
    """Main synthetic dataset generation pipeline"""
    
    def __init__(self, config):
        self.config = config
        self.scope = SCOPEHandler(config['scope_path'])
        self.modtran = MODTRANHandler(
            config['modtran_path'],
            config['modtran_template']
        )
        self.output_dir = Path(config['output_path']).resolve()
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def generate_dataset(self, num_samples=1000):
        """Generate complete synthetic dataset"""
        param_ranges = {
            'lai': (0.5, 6.0),
            'cab': (20, 80),
            'sza': (20, 70),
            'visibility': (5, 50),
            'sensor_altitude': (350, 1500)
        }

        for i in range(num_samples):
            params = self._random_parameters(param_ranges)
            try:
                # Run SCOPE simulation
                with self.scope as scope:
                    scope_result = scope.run_simulation(params)
                
                # Run MODTRAN simulation
                modtran_result = self.modtran.run_simulation(params, i)
                
                # Calculate final radiance
                radiance = self._calculate_radiance(
                    scope_result['reflectance'],
                    scope_result['fluorescence'],
                    modtran_result
                )
                
                # Save sample
                self._save_sample(i, radiance, params)

            except Exception as e:
                print(f"Error generating sample {i}: {str(e)}")
                continue

    def _random_parameters(self, ranges):
        """Generate random parameters within specified ranges"""
        return {
            'lai': np.random.uniform(*ranges['lai']),
            'cab': np.random.uniform(*ranges['cab']),
            'sza': np.random.uniform(*ranges['sza']),
            'visibility': np.random.uniform(*ranges['visibility']),
            'sensor_altitude': np.random.uniform(*ranges['sensor_altitude'])
        }

    def _calculate_radiance(self, reflectance, fluorescence, modtran):
        """Calculate at-sensor radiance using physics model"""
        return (
            (modtran['direct_irradiance'] * modtran['direct_transmittance'] +
            modtran['diffuse_irradiance'] * modtran['diffuse_transmittance'])
            * reflectance / (1 - reflectance * modtran['spherical_albedo'])
        ) + fluorescence * modtran['fluorescence_transmittance']

    def _save_sample(self, idx, radiance, params):
        """Save sample as compressed numpy file"""
        np.savez_compressed(
            self.output_dir / f'sample_{idx:04d}.npz',
            radiance=np.tile(radiance, (17, 17, 1)),  # Create 17x17 patch
            parameters=params,
            wavelength=self.scope.wavelength
        )